## Fresnel Transfer Function Approach

1. Given wavelength ($\lambda$),z, and the largest width of the aperture $l$, calculate the Fresnel Number $N_f$ appropriate for the entire aperture.
2. Choose an acceptable level of aliasing at the edge of the diffraction pattern.
3. Start by assuming that the aperture is rectangular, even though it may not be. Choose M, Q, and N=QM based on Figure 5.5
4. Create the padded aperture array of size NxN centered at index (N/2, N/2)
5. Create the quadratic-phase exponential transfer function array of length NxN, centered at index (N/2, N/2)
6. Perform a DFT on the aperture array, using an FFT algorithm. Center the result at index (N/2, N/2)
7. Multiply the two spectral arrays together element by element and perform an inverse DFT on the product, again using the FFT algorithm.
8. Circularly shift the center of the resulting sequence from indices (0,0) to indicies (N/2, N/2)
9. If intensity is of interest, take the squared magnitude of each element in the result of the inverse DFT. 
10. Experiment by increasing or decreasing M to determine its minimum allowable value that will satisfy the aliasing criterion. Choose Q=M/(4Nf) and adjust this value if necessary to yield good results.

In [ ]:
#Import some packages that we will need
import torch
import matplotlib.pyplot as plt
import pint
import numpy as np
import cv2

u = pint.UnitRegistry()

#plt.style.use(['science','notebook'])

%matplotlib ipympl

# Experiment 1: Single Slit Aperture

In [ ]:
#Now let's define our aperture width, our wavelength, and our propagation distance

#Aperture shape
l = 0.1 * u.mm
l_ = 0.5 * u.mm

#Wavelength
wavelength = 660 * u.nm

#Propagation Distance
z = 3 * u.cm

#Wavenumber (k)
wavenumber = 2*np.pi/wavelength

In [ ]:
#1) Find our Fresnel Number
N_f = ( (l/2)**2 ) / (wavelength * z)
N_f = N_f.to_base_units()

if N_f < 0.25:
    print(f"Nf = {N_f.magnitude} - We are in the Far Field")
else:
    print(f"Nf = {N_f.magnitude} - We are in the Near Field")

In [ ]:
#2) Choose acceptable aliasing
aliasing_error = 1e-3 #Pretty arbitrary

In [ ]:
#3) M, Q, N

#Samples in the open aperture
M = 15

#Sampling Ratio
Q = M / (4*N_f)
Q = Q.to_base_units()

#Total width of the aperture
L = wavelength * z * M / l
L = L.to_compact()

#Total number of samples in the simulation
N = Q * M

#Sample spacing in the aperture plane
dx = L / N
dy = dx
#Sample spacing in the Frequency domain
dfx = 1 / L
dfy = dfx

print(f"M = {M}")
print(f"Q = {Q}")
print(f"N = {N}")
print(f"L = {L}")

In [ ]:
#4) Create the NxN padded aperture array centered at (N/2, N/2)
# we will embed the open aperture in this array
x = np.linspace((-L/2).magnitude, (L/2).magnitude, int(N.to_base_units().magnitude)) *u.mm
y = x
xx,yy = np.meshgrid(x,y)

#Embed an aperture in the array
U0 = (np.abs(xx) < l / 2) * (np.abs(yy) < l_)
U0 = U0.astype(float)


In [ ]:
#Plot it to see the aperture
plt.figure(figsize=(5,5))
plt.pcolormesh(xx,yy,U0)
plt.xlabel('X-Position [mm]')
plt.ylabel('Y-Position [mm]')
plt.title("Single Slit Aperture")
plt.show()

In [ ]:
#5) Create the quadratic-phase exponential transfer function array 
kx = np.fft.fftfreq(len(x), np.diff(x)[0]) * 2 * np.pi
ky = kx
kxv, kyv = np.meshgrid(kx,ky)

H = np.exp(1j * wavenumber * z) * np.exp(-1j * 0.5 * z * wavelength / (2* np.pi) * (kxv**2 + kyv**2))
H = np.fft.fftshift(H.magnitude)

In [ ]:
#6) Perform the DFT of the aperture and then center
A = np.fft.fft2(U0)
A = np.fft.fftshift(A)

In [ ]:
plt.figure(figsize=(5,5))
plt.pcolormesh(np.fft.fftshift(kxv.magnitude), np.fft.fftshift(kyv.magnitude), np.abs(A))
plt.xlim(-100,100)
plt.ylim(-100,100)
plt.xlabel('$k_x$ [mm$^{-1}$]')
plt.ylabel('$k_y$ [mm$^{-1}$]')
plt.title("Spatial Frequency Spectrum \n of Aperture")
plt.tight_layout()
plt.show()

In [ ]:
#7) Multiply
U_ = A * H
U_ = np.fft.ifft2(U_)

In [ ]:
plt.figure(figsize=(5,5))
plt.pcolormesh(xx,yy,np.abs(U_), cmap='inferno')
plt.xlabel('$x$ [mm]')
plt.ylabel('$y$ [mm]')
plt.title("Aperture Diffraction Pattern")
plt.tight_layout()
plt.show()

In [ ]:
#Just a quick comparison to an experimental diffraction pattern
#Source: https://personal.math.ubc.ca/~cass/courses/m309-03a/m309-projects/krzak/
pattern = cv2.imread('../images/ss_diffraction_pattern.jpg')
pattern = cv2.cvtColor(pattern, cv2.COLOR_BGR2RGB)
pattern = np.asarray(pattern)

plt.figure(figsize=(5,5))
plt.imshow(pattern)
plt.axis('off')
plt.title("Experimental Diffraction Pattern")
plt.tight_layout()
plt.show()

# Experiment 2: Double Slit Aperture

In [ ]:
#Aperture dims
S = 0.2*u.mm
D = 0.05*u.mm

#Wavelength
wavelength = 660 * u.nm

#Propagation Distance
z = 3 * u.cm

#Wavenumber (k)
wavenumber = 2*np.pi/wavelength

In [ ]:
#1) Find our Fresnel Number
N_f = ( (D/2)**2 ) / (wavelength * z)
N_f = N_f.to_base_units()

if N_f < 0.25:
    print(f"Nf = {N_f.magnitude} - We are in the Far Field")
else:
    print(f"Nf = {N_f.magnitude} - We are in the Near Field")

In [ ]:
#3) M, Q, N

#Samples in the open aperture
M = 12

#Sampling Ratio
Q = M / (4*N_f)
Q = Q.to_base_units()

#Total width of the aperture
L = wavelength * z * M / D
L = L.to_compact()

#Total number of samples in the simulation
N = Q * M

#Sample spacing in the aperture plane
dx = L / N
dy = dx
#Sample spacing in the Frequency domain
dfx = 1 / L
dfy = dfx

print(f"M = {M}")
print(f"Q = {Q}")
print(f"N = {N}")
print(f"L = {L}")

In [ ]:
#4) Create the NxN padded aperture array centered at (N/2, N/2)
# we will embed the open aperture in this array
x = np.linspace((-L/2).magnitude, (L/2).magnitude, int(N.to_base_units().magnitude)) *u.mm
y = x
xx,yy = np.meshgrid(x,y)

U0 = (np.abs(xx-S/2)< D/2) * (np.abs(yy)<2*u.mm) + (np.abs(xx+S/2)< D/2) * (np.abs(yy)<2*u.mm)
U0 = U0.astype(float)

In [ ]:
plt.figure(figsize=(5,5))
plt.pcolormesh(xx,yy,U0)
plt.xlabel('X-Position [mm]')
plt.ylabel('Y-Position [mm]')
plt.tight_layout()
plt.show()

In [ ]:
#5) Create the quadratic-phase exponential transfer function array 
kx = np.fft.fftfreq(len(x), np.diff(x)[0]) * 2 * np.pi
ky = kx
kxv, kyv = np.meshgrid(kx,ky)

H = np.exp(1j * wavenumber * z) * np.exp(-1j * 0.5 * z * wavelength / (2* np.pi) * (kxv**2 + kyv**2))
H = np.fft.ifftshift(H.magnitude)

In [ ]:
#6) Perform the DFT of the aperture and then center
A = np.fft.fft2(U0)
A = np.fft.fftshift(A)

In [ ]:
plt.figure(figsize=(5,5))
plt.pcolormesh(np.fft.fftshift(kxv.magnitude), np.fft.fftshift(kyv.magnitude), np.abs(A))
plt.xlim(-100,100)
plt.ylim(-100,100)
plt.xlabel('$k_x$ [mm$^{-1}$]')
plt.ylabel('$k_y$ [mm$^{-1}$]')
plt.title("Spatial Frequency Spectrum \n of Aperture")
plt.tight_layout()
plt.show()

In [ ]:
#7) Multiply
U_ = A * H
U_ = np.fft.ifft2(U_)

In [ ]:
plt.figure(figsize=(5,5))
plt.pcolormesh(xx,yy,np.abs(U_), cmap='inferno')
plt.xlabel('X-Position [mm]')
plt.ylabel('Y-Position [mm]')
plt.title("Aperture Diffraction Pattern")
plt.tight_layout()
plt.show()

# Experiment 3: Hexagonal Grating

In [ ]:
img = cv2.imread('../images/hexagon_grating.jpg')
img = np.pad(img, 200, mode='constant')

l = 0.1*u.mm #Approximate width of hexagon element

#Wavelength
wavelength = 660 * u.nm

#Propagation Distance
z = 3 * u.cm

#Wavenumber (k)
wavenumber = 2*np.pi/wavelength

In [ ]:
#1) Find our Fresnel Number
N_f = ( (l/2)**2 ) / (wavelength * z)
N_f = N_f.to_base_units()

if N_f < 0.25:
    print(f"Nf = {N_f.magnitude} - We are in the Far Field")
else:
    print(f"Nf = {N_f.magnitude} - We are in the Near Field")

In [ ]:
#3) M, Q, N

#Samples in the open aperture
M = 15

#Sampling Ratio
Q = M / (4*N_f)
Q = Q.to_base_units()

#Total width of the aperture
L = wavelength * z * M / l
L = L.to_compact()

#Total number of samples in the simulation
N = Q * M

#Sample spacing in the aperture plane
dx = L / N
dy = dx
#Sample spacing in the Frequency domain
dfx = 1 / L
dfy = dfx

print(f"M = {M}")
print(f"Q = {Q}")
print(f"N = {N}")
print(f"L = {L}")

In [ ]:
#4) Create the NxN padded aperture array centered at (N/2, N/2)
# we will embed the open aperture in this array
x = np.linspace((-L/2).magnitude, (L/2).magnitude, int(N.to_base_units().magnitude)) *u.mm
y = x
xx,yy = np.meshgrid(x,y)

img = cv2.resize(img, dsize=(len(x), len(y)), interpolation=cv2.INTER_CUBIC)
U0 = np.array(img).sum(axis=2).astype(float)

In [ ]:
#Plot it to see the aperture
plt.figure(figsize=(5,5))
plt.pcolormesh(xx,yy,U0)
plt.xlabel('X-Position [mm]')
plt.ylabel('Y-Position [mm]')
plt.title("Hexagonal Grating")
plt.tight_layout()
plt.show()

In [ ]:
#5) Create the quadratic-phase exponential transfer function array 
kx = np.fft.fftfreq(len(x), np.diff(x)[0]) * 2 * np.pi
ky = kx
kxv, kyv = np.meshgrid(kx,ky)

H = np.exp(1j * wavenumber * z) * np.exp(-1j * 0.5 * z * wavelength / (2* np.pi) * (kxv**2 + kyv**2))
H = np.fft.fftshift(H.magnitude)

In [ ]:
#6) Perform the DFT of the aperture and then center
A = np.fft.fft2(U0)
A = np.fft.fftshift(A)

In [ ]:
plt.figure(figsize=(5,5))
plt.pcolormesh(np.fft.fftshift(kxv.magnitude), np.fft.fftshift(kyv.magnitude), np.abs(A))
plt.xlim(-100,100)
plt.ylim(-100,100)
plt.xlabel('$k_x$ [mm$^{-1}$]')
plt.ylabel('$k_y$ [mm$^{-1}$]')
plt.title("Spatial Frequency Spectrum \n of Hexagonal Grating")
plt.tight_layout()
plt.show()

In [ ]:
#7) Multiply
U_ = A * H
U_ = np.fft.ifft2(U_)

In [ ]:
plt.figure(figsize=(5,5))
plt.pcolormesh(xx,yy,np.abs(U_), cmap='inferno')
plt.xlabel('X-Position [mm]')
plt.ylabel('Y-Position [mm]')
plt.title("Hexagonal Grating \nDiffraction Pattern")
plt.tight_layout()
plt.show()

In [ ]:
with open('hexahonal_dp.npy', 'wb') as f:
    np.save(f, U_)